In [1]:
import basis as bs
import mesh as mm
import integrate as ig
import assembly as asm
import numpy.polynomial.legendre as lg
import numpy as np

In [2]:
class generate_mesh():
    def __init__(self, phys_xmin, phys_xmax, n_elem):
        self.phys_xmin = phys_xmin
        self.phys_xmax = phys_xmax
        self.n_elem = n_elem
        
        self.gen_all_nodes()
        self.gen_elem_node_coord_indices()

    def gen_all_nodes(self):
        self.all_nodes = np.linspace(self.phys_xmin, self.phys_xmax, self.n_elem + 1)
        return self.all_nodes
    def gen_elem_node_coord_indices(self):
        arr = np.zeros((2, self.n_elem), dtype=int)
        for n in range(0, self.n_elem):
            arr[0,n] = n
            arr[1,n] = n+1
        self.elem_node_coord_indices = arr
        return self.elem_node_coord_indices
    def get_element_domain(self, e):
        n0_idx = int(self.elem_node_coord_indices[0,e])
        n1_idx = int(self.elem_node_coord_indices[1,e])
        self.element_domain = np.array((self.all_nodes[n0_idx], self.all_nodes[n1_idx]))
        return self.element_domain

In [3]:

def gen_ke(mesh, e):
    # for e in range(0, self.n_elem):
    # mx = bs.stock_mx()
    mx_ = np.zeros((2,2))
    e_dom = mesh.get_element_domain(e)
    e_x0 = e_dom[0]
    e_x1 = e_dom[1]
    mx_mult = bs.mxm(e_x0, e_x1)
    inv_term = bs.x_xi_deriv_inv(e_x0, e_x1)
    for a in range(0,2):
        N_a_xi_term = bs.N_a_xi(a)
        for b in range(0,2):
            N_b_xi_term = bs.N_a_xi(b)
            ke_entry_incr = N_a_xi_term * N_b_xi_term * inv_term
            mx_[a, b] += ke_entry_incr
    ke = mx_
    return ke

In [12]:
m1 = generate_mesh(0, 1, 1)

e_dom = m1.get_element_domain(0)
e_x0 = e_dom[0]
e_x1 = e_dom[1]
# print(f"{e_x0}, {e_x1}")
mult = bs.mxm(e_x0, e_x1)
inv_term = bs.x_xi_deriv_inv(e_x0, e_x1)
# print(inv_term)
# gen_ke(m1, 0)
N_ab_xi_term = bs.N_a_xi(1) * bs.N_a_xi(1)
ke_entry_incr = N_ab_xi_term * inv_term

0.5

In [17]:
def N_a_xi(idx):
    neg = (-1)**idx
    return neg / 2
# THIS RETURNS POS/NEG 0.5, NOT 1

def Nab(a,b):
    return N_a_xi(a) * N_a_xi(b)
# SO WITH TWO OF THEM, RETURNS POS/NEG 0.25
# Nab(0,0) # +0.25
# Nab(0,1) # -0.25


0.25

In [25]:
# ADD IN INVERSE X-XI MAP TERM:
def x_xi_deriv_inv(e_x0, e_x1): 
    h_e = e_x1 - e_x0
    return 2 / h_e
# AND FOR ONE ELEMENT ON 0,1, WE GET:
inv_term = x_xi_deriv_inv(0,1) # 2
print(inv_term)
# SO WITH THE ABOVE, WE HAVE A PRODUCT OF POS/NEG 0.5
prod = x_xi_deriv_inv(0,1) * Nab(0,1)
print(prod)
# BUT THIS SHOULD COME OUT TO THE SAME AS THE STOCK MATRIX x 1/H_E

2.0
-0.5


In [26]:
# BUT WE ALSO HAVE TO INTEGRATE: WE DON'T GET STOCK MATRIX x 1/H_E FROM THE ABOVE PRODUCT. WE GET IT FROM EVALUATING THAT PRODUCT'S INTEGRAL FROM -1 TO 1

upper = prod * 1 
lower = prod * -1
upper - lower

# HOLY CRAP THAT'S IT. 
# NOW, GAUSS QUADRATURE SHOULD YIELD THE SAME THING:

-1.0